# Анализ рынка продаж компьютерных игр

Описание проекта: 
- У нас есть данные интернет-магазина «Стримчик», который продаёт по всему миру компьютерные игры.
- Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). 
- Нам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.
- Перед вами данные до 2016 года. Планируется кампания на 2017-й. Нужно отработать принцип работы с данными.
- В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

Описание данных:

- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы долларов)
- EU_sales — продажи в Европе (миллионы долларов)
- JP_sales — продажи в Японии (миллионы долларов)
- Other_sales — продажи в других странах (миллионы долларов)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.



In [ ]:
# импортируем потенциально необходимые нам библиотеки

import pandas as pd
import numpy as np
import scipy
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from scipy import stats as st
from scipy.stats import levene
import math
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots


# Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
try:
    game_data = pd.read_csv("/datasets/games.csv")  

except:
    print("Есть ошибки")
else:
    print("Все загрузилось!")

In [ ]:
# посчитаем количество дубликатов
game_data.duplicated().sum()

Дубликатов нет.

In [ ]:
# посчитаем количество пустых значений
game_data.isna().sum()

In [ ]:
# посмотрим на столбцы:
game_data.columns

In [ ]:
# отобразим таблицу, где имеются пустые значения в столбце Сritic_Score:
game_data[game_data['Critic_Score'].isna()]

In [ ]:
# отобразим таблицу, где имеются пустые значения в столбце user_score:
game_data[game_data['User_Score'].isna()]

In [ ]:
# отобразим таблицу, где имеются пустые значения в столбце Rating:
game_data[game_data['Rating'].isna()]

In [ ]:
# отобразим таблицу, где имеются пустые значения в столбце Year_of_release:
game_data[game_data['Year_of_Release'].isna()]

In [ ]:
# отобразим платформы:
game_data['Platform'].value_counts()

In [ ]:
# отобразим жанры:
game_data['Genre'].value_counts()

Выводы:

- Необходимо внести изменения в данные столбца "Year of Release".
- Также нужно привести к нижнему регистру названия колонок и можно данные столбцов.
- Имеются пустые значения в столбцах rating, user_score, critic_score, name,year_of_release,genre.
- Дубликатов нет.

# Шаг 2. Подготовьте данные

Задание:
- Замените названия столбцов (приведите к нижнему регистру);
- Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
- Обработайте пропуски при необходимости:
- Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
- Опишите причины, которые могли привести к пропускам;
- Обратите внимание на аббревиатуру 'tbd' в столбце с оценкой пользователей. Отдельно разберите это значение и опишите, как его обработать;
- Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

In [ ]:
# приведем к нижнему регистру наименования столбцов
game_data.columns = map(str.lower,game_data.columns)

In [ ]:
# выведем обновленные наименования столбцов( для проверки обновления)
list(game_data.columns.values)

In [ ]:
# приведем к нижнему регистру следующие столбцы
for column in game_data[['name','platform','genre']]:
    game_data[column] = game_data[column].str.lower()

In [ ]:
# выведем первые 10 строк таблицы:
display(game_data.head(10))

Столбцы platform, na_sales, eu_sales, jp_sales,other_sales содержат одинаковое количество строк: 16715.

In [ ]:
game_data.shape

In [ ]:
game_data.describe(include = "all")

In [ ]:
# удаляем строки с пустыми значениями в столбцах
game_data.dropna(subset = ['name', 'year_of_release', 'genre'], inplace = True)

In [ ]:
game_data.isna().sum()

In [ ]:
# изменим тип данных в столбце "year_of_release", так годы выпуска выглядят более корректно
game_data['year_of_release'] = game_data['year_of_release'].astype('Int64')

In [ ]:
game_data.info()

In [ ]:
# заполняем столбцы critic_score, user_score медианой 
game_data.loc[game_data['critic_score'].isna(), 'critic_score'] = -1
game_data.loc[game_data['user_score'].isna(), 'user_score'] = -1

Мы не можем заменить NaN в рейтинге с данными.

In [ ]:
# Проверим каждое значение "рейтинге"
game_data.rating.value_counts()

In [ ]:
game_data.rating.isnull().sum()

In [ ]:
# так сейчас выглядят данные таблицы
game_data.sample(5)

Аббревиатура tbd = to be determined, to be done. 
Данные были не заполнены, можно заменить "tbd" на "nan".

In [ ]:
#заменим tbd на nan
game_data['user_score'] = game_data['user_score'].replace('tbd', np.nan, regex=True)

In [ ]:
#поменяем формат столбца user_score на float
game_data['user_score'] = game_data['user_score'].astype(float)
game_data['user_score'].dtype

Посчитаем суммарные продажи во всех регионах и запишем их в отдельный столбец "total_sales":

In [ ]:
#создадим новый столбец "total_sales"
game_data['total_sales'] = game_data['na_sales'] + game_data['eu_sales'] + game_data['jp_sales'] + game_data['other_sales']

In [ ]:
list(game_data.columns.values)

In [ ]:
# можно заполнить значения с пометкой NaN постоянным значением, эти значения теперь являются фактическими значениями, а не NaN .
game_data['rating'].fillna('нет данных', inplace=True) 

In [ ]:
game_data.sample(5)

Основные выводы по подготовке данных:

- Привели к нижнему регистру все столбцы таблицы.
- Привели к корректному типу данных столбец "year of release".
- Дубликатов не выявлено.
- Поменяли значение "tbd" столбца 'user_score'на nan.
- Убрали пустые значения в столбцах 'name', 'year_of_release', 'genre'.
- Заполнили столбцы "critic_score", "user_score" медианой, поменяли формат столбца "user_score" на "float".
- Данные по суммарным продажам вынесены в столбик "total_sales".

# Шаг 3. Проведите исследовательский анализ данных

Задание:

- Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
- Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
- Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год.
- Не учитывайте в работе данные за предыдущие годы.
- Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.
- Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.
- Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.
- Соотнесите выводы с продажами игр на других платформах.
- Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?

In [ ]:
# перечень платформ
game_data['platform'].value_counts()

In [ ]:
#сколько игр выпускалось в разные годы - 'year_of_release'
game_data['year_of_release'].value_counts()

In [ ]:
game_data['year_of_release'].hist()

In [ ]:
# создадим сводную таблицу на основе year_of_release:
game_year = game_data.pivot_table(index = 'year_of_release', values= 'name', aggfunc='count' ).reset_index()
game_year.columns = ['year', 'games_count']
game_year = game_year.sort_values('games_count', ascending=False)
game_year.head(5)

In [ ]:
fig = px.bar(game_year, x = 'year', y = 'games_count',
             title='Гистограмма выпуска игр по каждому году',
            labels=dict(year='Годы', games_count="Количество игр"))

fig.show()

По графику видно, количество игр начало расти с 1990-х годов по 2008 год, затем последовал некторый спад. 
Данные за весь период не важны, так как есть периоды с минимальными показателями.Для анализа и прогноза на 2017 год
целесообразно выделить годы, по которым можно будет анализировать динамику продаж.
Выберем платформы с наибольшими продажами и построим распределение по годам, актуальный период определим самостоятельно.Посмотрим как менялись продажи по платформам. 
Выберем платформы с наибольшим общим объемом продаж и построим данные по продажам за каждый год.

In [ ]:
top_10 = game_data.pivot_table(index = ['platform'], values = 'total_sales', aggfunc = 'sum').reset_index().sort_values("total_sales", ascending=False)
top_10 = top_10.nlargest(10, 'total_sales')
top_10.head(10)

Построим сводную таблицу данных для общего объема продаж по каждой платформе 

In [ ]:
top_list = top_10['platform'].tolist()
only_top = game_data.query('platform in @top_list')
platform = only_top.pivot_table(index = ['platform','year_of_release' ], values = 'total_sales', aggfunc = 'sum').reset_index()
platform.head()

In [ ]:
#продажи платформы за каждый год
fig = px.scatter(platform, x='year_of_release', y='platform',
                 title='Продажи платформ по годам',
            labels=dict(year_of_release='Годы', platform="Платформа",
             height=500))
fig.show()

Из  диаграммы мы можем видеть жизненный цикл платформы:
PS, PS2, GBA и DS - раньше были востребованы, X360, Wii, PS3 и PS4 - присутствуют на рынке.

In [ ]:
fig = px.area(platform, x="year_of_release", y="total_sales", color="platform",
               title='Продажи по каждой платформе',
            labels=dict(year_of_release='Годы', platform="Platform",total_sales="Продажи" ),
              line_group="platform",
             height=600)
fig.show()

Жизненный цикл игрового продукта - около 10 лет. 

Для планирования и прогнозирования продаж  на 2017 год важно использовать только релевантные данные за 2014-2016гг.:

In [ ]:
# отфильтруем годы после 2014
filtered= game_data.query('year_of_release >= 2014')
filtered.sort_values(by='total_sales')
filtered.head()

In [ ]:
#сводная таблица для определения общего объема продаж на каждой платформе по годам
game_year = pd.pivot_table(filtered, index='year_of_release', columns='platform', values='total_sales', aggfunc=sum, fill_value=0)
game_year.head()

In [ ]:
# cравним:
trend = game_year
trend.head(5)

In [ ]:
# построим тепловую карту продаж платформ за период 2014-2016гг.

fig = px.imshow(trend,
                labels=dict(year_of_release="Year", platform="Platform"),
                title='Продажи платформ за период 2014-2016 гг.',
                height=300)

fig.show()

Тепловая карта показывает:
Платформа PS4 была очень популярна в 2014-2015гг., но в 2016 году упала.
Xbox One занимает второе место по продажам в 2014 году и с тех пор начала постепенно падать.
У Xbox 360 в 2014 году продажи упали, но в 2015-2016 годах немного выровнялись.
Продажи Wii немного снизились в 2014 года.
Продажи PS3 просели в 2014 году, а PS4 стала популярной.

In [ ]:
# сгруппируем данные по годам и платформе с общей суммой продаж
avg_sale = filtered.groupby(['platform'])['total_sales'].sum().reset_index()
avg_sale.sort_values('total_sales', ascending=True)

Топ-5 платформ ( с выручкой более 100 млн):

- PS4 (314,14)
- PS3 (288,79)
- X360 (236,54) 
- 3DS (194,61) 
- XOne (159,323)


In [ ]:
# сгруппируем данные по годам, платформе с общей суммой продаж:
total_sale = filtered.groupby(['platform','year_of_release'])['total_sales'].sum().reset_index()
total_sale.sample(5)

In [ ]:
# построим график box-and-whisker plot: 
fig = px.box(total_sale, x="platform", y="total_sales", color='platform',
            title='Средние продажи на различных платформах 2014-2016гг.',
            labels=dict(total_sales='Total Sales', platform="Platform"),
            height=600).update_xaxes(categoryorder = 'total descending')
fig.show()

PS4 является лидером рынка по среднему общему объему продаж, за ней следуют  xone, 3DS, ps3.

Как отзывы пользователей и профессионалов влияют на продажи платформы. Для этого проанализируем, например, xone.

In [ ]:
# выберем для анализа xone
xone = filtered.query(' platform == "xone"')
xone.head()

In [ ]:
# Посмотрим на графическое представление продаж xone и данных по отзывам пользователей и критиков
fig, axs = plt.subplots(ncols=2, figsize=(18, 7))
sns.regplot(x="user_score", y="total_sales", data=xone, ax=axs[0], color='b')
sns.regplot(x="critic_score", y="total_sales", data=xone, ax=axs[1], color='r')
axs[0].set_title('Отзывы пользователей и продажи 2014-2016 (xone)', fontsize=11, color='b')
axs[1].set_title('Отзывы критиков и продажи 2014-2016 (xone)', fontsize=11, color='r')
plt.show()

In [ ]:
xone_corr = xone.pivot_table(index='total_sales', values=['user_score','critic_score']).reset_index()
xone_corr.corr()

Мы построили диаграмму рассеяния, корреляция между отзывами критиков, пользователей и продажами посчитана. 
Оценка критиков xone имеет корреляцию с общим объемом продаж, в то время как оценка пользователей - меньше.

Сравним продажи одних и тех же игр на других платформах

In [ ]:
selected_plat = filtered.query(' platform in ("ps3", "ps4", "sds")')

In [ ]:
other_plat = selected_plat.pivot_table(index='total_sales', values=['user_score','critic_score']).reset_index()
other_plat.tail()

In [ ]:
# Построим диаграмму рассеяния (оценки пользователей и оценки критиков)
fig, axs = plt.subplots(ncols=2, figsize=(18, 6))
sns.regplot(x="user_score", y="total_sales", data=other_plat, ax=axs[0], color='b')
sns.regplot(x="critic_score", y="total_sales", data=other_plat, ax=axs[1], color='r')
axs[0].set_title('Отзывы пользователей и продажи  2014-2016 (Top platforms)', fontsize=10, color='b')
axs[1].set_title('Отзывы критиков и продажи 2014-2016 (Top platforms)', fontsize=10, color='r')
plt.show()

In [ ]:
other_plat.corr()

Оценка критиков оказывает все-таки незначительное влияние на общий объем продаж, в то время как оценка пользователей не оказывает.

Посмотрим на общее распределение игр по жанрам. 

In [ ]:
# выберем платформы
pop_plat = filtered.query(' platform in ("ps4","ps3", "3ds", "xone")')

In [ ]:
# соберем сводную информацию об общем объеме продаж, сгруппированную по платформам
genre_pivot = pop_plat.pivot_table(index=['platform', 'genre'],
                                          values=['na_sales','eu_sales','jp_sales' ,'other_sales' ,'total_sales' ],
                                          aggfunc=sum).reset_index()\
.sort_values(by='total_sales', ascending=False)

In [ ]:
#построим гистограмму общего объема продаж в каждом жанре по платформам
fig = px.bar(genre_pivot, x="genre", y='total_sales', barmode='group',
             color='platform',
             title='Общие продажи игр по жанрам 2014-2016гг.',
             labels=dict(total_sales='Total Sales', genre='Genre'),
             height=600)

fig.update_layout(barmode='stack')
fig.show()

In [ ]:
# создадим сводные данные общих продаж, сгруппированные по жанрам
genre_comp = filtered.pivot_table(index=['genre'],
                                       values=['na_sales','eu_sales','jp_sales' ,'total_sales','other_sales' ],
                                       aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

genre_comp.sample(5)

- Пять самых популярных жанров на гистограмме: Action,Shooter,Sports,Role-Playing,Racing.
- Role-Playing, action,spors,shooter популярны в ps4, в то время как Racing - меньше. Strategy and Puzzle не пользуются популярностью в течение 2014-2016 годов.
- PS4, xone,3ds получили продажи от большинства игровых жанров.

# Шаг 4. Составьте портрет пользователя каждого региона

Задание:

Определите для пользователя каждого региона (na, eu,jp):
- Самые популярные платформы (топ-5). Опишите различия в долях продаж.
- Самые популярные жанры (топ-5). Поясните разницу.
- Влияет ли рейтинг ESRB на продажи в отдельном регионе?

Справочная информация по аббревеатурам "Entertainment Software Rating Board" (ESRB):

- «E» («Everyone») — «Для всех»

- «E10+» («Everyone 10 and older») — «Для всех от 10 лет и старше»

- «T» («Teen») — «Подросткам»

- «M» («Mature») — «Для взрослых»

- «AO» («Adults Only 18+») — «Только для взрослых»

- «RP» («Rating Pending») — «Рейтинг ожидается»

- «K-A» («Kids to Adults»)- 2-я версия 1996 - 1997 гг. = «E» («Everyone») — «Для всех»

- «EC» («Early childhood») — «Для детей младшего возраста»

Источник: https://www.esrb.org/ratings/

In [ ]:
#данные по общим продажам по платформам
region = filtered.pivot_table(index=['platform'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='total_sales', ascending=False)

#select only the top five in total sale
top_plat_region = region.head(5)
top_plat_region

In [ ]:
#построим диаграмму региональных продаж по каждой платформе
fig = px.bar(top_plat_region, x="platform", y=['eu_sales','jp_sales', 'na_sales','other_sales'], barmode='group',
             title='Общие продажи платформ по регионам 2014-2016гг.',
            labels=dict(value='Total Sales', platform='Platform'),
             height=500)

fig.update_layout(barmode='stack')
fig.show()

PS4 лидирует по общему объему продаж в большинстве регионов, за исключением Японии.
Xone является  популярным в Северной Америке, но получил низкие продажи в Японии.
3DS очень популярна в Японии, в то время как xone, x360 получила самые низкие продажи в Японии.

In [ ]:
#построим диаграмму "Доля рынка: между платформами в каждом регионе"

labels = top_plat_region['platform']

fig = make_subplots(rows=1, cols=4, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['eu_sales'], name="Евросоюз"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['jp_sales'], name="Япония"),
              1, 2)
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['na_sales'], name="Северная Америка"),
              1, 3)
fig.add_trace(go.Pie(labels=labels,values=top_plat_region['other_sales'], name="Другие"),
              1, 4)


fig.update_layout(
    title_text="Рыночная доля каждого региона 2014-2016гг.",

    annotations=[dict(text='Евросоюз', x=0.02, y=0.9, font_size=15, showarrow=False),
                 dict(text='Япония', x=0.36, y=0.9, font_size=15, showarrow=False),
                dict(text='Северная Америка', x=0.76, y=0.9, font_size=15, showarrow=False),
                dict(text='Другие', x=0.97, y=0.9, font_size=15, showarrow=False)])
fig.show()

- В Евросоюзе самая высокая доля PS4, за ней следует xone, в то время как у x 360 самая низкая доля.
- У Японии есть один лидер 3ds, за ним следуют PS3 и PS4, в то время как Xbox 360 и xone не пользуются популярностью.
- В Северной Америке большинство топовых платформ получили очень похожее соотношение. Самый высокий показатель - PS4, xone, затем x360, 3ds и ps3 почти одинаково распределены.
- Другие: лидером рынка является PS4. На втором месте xone. Наименьшая доля на 3DS.

In [ ]:
# топ 5 по регионам
genre = filtered.pivot_table(index=['genre'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index().sort_values(by='total_sales', ascending=False)

top_plat_genre = genre.head(5)
top_plat_genre

In [ ]:
# представим данные  по жанрам графически

fig = px.bar(top_plat_genre, x="genre", y=['eu_sales','jp_sales', 'na_sales','other_sales'], barmode='group',
             title='Общие продажи по жанрам в регионах 2014-2016 гг.',
            labels=dict(value='Total Sales', genre='Genre'),
             height=600)

fig.update_layout(barmode='stack')
fig.show()

- Action является лидером по общему объему продаж  на рынке, с самыми высокими продажами в Северной Америке и Европе.
- Shooter, Sports  не популярен в Японии,самые высокие продажи в жанре role-playing.
- Misc не пользуется популярностью в большинстве регионов.

In [ ]:
# построим диаграмму" Доля рынка по жанрам в каждом регионе"
labels = top_plat_genre['genre']

fig = make_subplots(rows=1, cols=4, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['eu_sales'], name="Евросоюз"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['jp_sales'], name="Япония"),
              1, 2)
fig.add_trace(go.Pie(labels=labels, values=top_plat_region['na_sales'], name="Северная Америка"),
              1, 3)
fig.add_trace(go.Pie(labels=labels,values=top_plat_region['other_sales'], name="Другие"),
              1, 4)


fig.update_layout(
    title_text="Доля рынка по регионам 2014-2016гг.",
  
    annotations=[dict(text='Евросоюз', x=0.02, y=0.9, font_size=20, showarrow=False),
                 dict(text='Япония', x=0.36, y=0.9, font_size=20, showarrow=False),
                dict(text='Северная Америка', x=0.76, y=0.9, font_size=20, showarrow=False),
                dict(text='Другие', x=0.97, y=0.9, font_size=20, showarrow=False)])
fig.show()

-  Евросоюз: На Action приходится самая высокая доля, а на misc- самая низкая.
-  Япония: Sport имеет самую высокую долю,  показатели misc на этом рынке очень низки.
-  Доля  Role-playing самая высокая в Японии.В других странах она несколько ниже. 
-  На рынке "Другие" самая популярная -  Action, на втором месте shooter, наименее популярная платформа - Sports.

Японский рынок отличается от других регионов самыми высокими продажами платформы 3DS и жанра role-playing.
За пределами Японии xone, PS4 входят в популярное большинство.
PS4 является лидером во всех регионах, кроме Японии.

Выясним влияет ли рейтинг ESRB на продажи в отдельном регионе.

In [ ]:
#сводные данные по продажам региона в рейтинге ESRB
grouped_rating = filtered.pivot_table(index=['rating'],
                                      values =['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales'],
                                      aggfunc=sum)\
.reset_index()\
.sort_values(by='total_sales', ascending=False)

grouped_rating

In [ ]:
#диаграмма общего объема продаж по регионам и рейтинг

fig = px.bar(grouped_rating, x="rating", y=['eu_sales','jp_sales', 'na_sales','other_sales'], barmode='group',
             title='Общие продажи в регионах и рейтинг за период 2014-2016гг.',
            labels=dict(value='Total Sales', rating='Rating'),
             height=500)

fig.update_layout(barmode='stack')
fig.show()

Вывод:
Наиболее распространенным на рынке является рейтинг M, "Нет данных", E,Т,E10+. Остальные не пользуются спросом и не отображаются.

# Шаг 5. Проверьте гипотезы

Задание:
    
- 5.1.Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
- 5.2.Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

Задайте самостоятельно пороговое значение alpha.
Поясните:
- Как вы сформулировали нулевую и альтернативную гипотезы;
- Какой критерий применили для проверки гипотез и почему.

5.1. 
- Нулевая гипотеза H₀: Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.
- Альтернативная гипотеза H₁: Средние пользовательские рейтинги платформ Xbox One и PC различные.

In [ ]:
# сохраним в переменных xone_h и pc_h соответствующие актуальные данные 

xone_h = game_data[(game_data['platform']=='xone') & ( game_data['year_of_release']>=2012)]['user_score']
pc_h = game_data[( game_data['platform']=='pc') & ( game_data['year_of_release']>=2012)]['user_score']

#посчитаем средний рейтинг пользователя для xbox платформ
xone_h.mean()

In [ ]:
# посчитаем средний рейтинг пользователя для PC платформ
pc_h.mean()

In [ ]:
# выполним проверку гипотезы. Будем использовать метод ttest_ind

alpha = .05

results = st.ttest_ind(xone_h.dropna(), pc_h.dropna(), equal_var=False)

print('p-значение:', results.pvalue)


if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Вывод: Таким образом, не получилось опровергнуть Нулевую гипотезу. 


5.2.   
Для проверки гипотезы "средние пользовательские рейтинги жанров Action и Sports разные" в качестве нулевой и альтернативной гипотезы мы взяли следующее:
H0: средние рейтинги по жанрам одинаковые.
H1: средние рейтинги по жанрам разные.
   

In [ ]:
# сохраним в переменных genre_action_h и genre_sports_h соответствующие данные с пользовательскими оценками

genre_action_h = game_data[(game_data['genre']=='action') & (game_data['year_of_release']>=2012)]['user_score']
genre_sports_h = game_data[(game_data['genre']=='sports') & (game_data['year_of_release']>=2012)]['user_score']

#выведем среднюю оценку по жанру экшн
genre_action_h.mean()

In [ ]:
genre_sports_h.mean()

In [ ]:
#выполним проверку гипотезы. Будем использовать метод ttest_ind

alpha = .05

results = st.ttest_ind(genre_action_h.dropna(), genre_sports_h.dropna(), equal_var=False)

print('p-значение:', results.pvalue)


if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Вывод: Получив p-value, мы отвергли Нулевую гипотезу. 
Таким образом, у нас практически нет вероятности получить одинакове средние рейнтинги по двум жанрам.

# Шаг 6. Общий вывод

- Цель этого проекта - выявить закономерности и составить прогноз.
- В целом региональные портреты пользователей по многим показателям и характеристикам совпадают, с некторыми небольшими различиями. При имеющихся данных можно использовать факторный метод прогнозирования продаж, где изначально устанавливаются  показатели прибыли предшествующих периодов и рассчитываются показатели на интересующий период и устанавливаются факторы, которые влияют на продажи. Для прогноза прибыли и продаж лучше брать данные за актульный период, они более актуальны со всех точек зрения(технологии, предпочтения пользователей можно "экстраполировать" более обосновано). 
- Мы подготовили данные согласно заданию: заменили названия столбцов (привели к нижнему регистру), преобразовали данные в нужные типы,обработали пропуски так, чтобы в массиве данных был одинаковый тип данных.
- Наш анализ охватывает региональный анализ рынков, профили ключевых платформ, спрос, направления роста.Анализ показал, что пользователи из разных регионов предпочитают разные жанры. 
- Таким образом, маркетинговая стратегия  должна ориентироваться  на данные регионов. Action, Shooters являются востребованными в Северной Америке и Европе, в то время как японский рынок сосредоточен на Sports. Role-playing на третьем месте по популярности во всех регионах.
- По каждой платформе есть разные тренды.Развитие должно быть быть нацелено на PS4,PS3, xone, у которой есть потенциал продаж и популярность, чтобы увеличить продажи в Северной Америке и Европе. Находясь в Японии, мы должны сосредоточиться на продвижении платформы 3DS, которая там наиболее популярна.
- Поскольку рейтинги M и E  популярны в большинстве регионов, лучше учитывать это при проведении рекламной камании, если мы хотим увеличить общий объем продаж.